In [ ]:
from ast import Index
from json import load
import os
import getpass
import time
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4

from pinecone import Pinecone, ServerlessSpec


if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = "recipes"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)
index = pc.Index(index_name)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

vector_store = PineconeVectorStore.from_existing_index(
    
    embedding=embeddings,
    index_name=index_name,
)

search = vector_store.similarity_search("tofu")

search
